In [1]:
import os
import rpy2.robjects as robjects

# Define the path to the data folder
data_path = "sessions"

# Function to read an RDS file
def read_rds(file_path):
    readRDS = robjects.r['readRDS']
    return readRDS(file_path)

In [2]:
session1_data = read_rds("sessions/session1.rds")
print(session1_data.names)

[1] "contrast_left"  "contrast_right" "feedback_type"  "mouse_name"    
[5] "brain_area"     "date_exp"       "spks"           "time"          



In [3]:
# Extract spike train matrix for the first trial
trial1_spks = np.array(session1_data.rx2("spks")[0])  # Shape: (n_neurons, n_time_bins)

# Print dimensions (equivalent to `dim()` in R)
print(f"Shape of spike train matrix for Trial 1: {trial1_spks.shape}")  
# Expected output: (734, 40) --> (n_neurons, n_time_bins)

NameError: name 'np' is not defined

In [ ]:
# Extract number of neurons
num_neurons = len(session1_data.rx2("brain_area"))
print(f"Number of neurons in Session 1: {num_neurons}")  
# Expected output: 734

In [ ]:
# Extract the spike train of neuron 6 (index 5 in Python, since indexing starts at 0)
neuron_6_spike_train = trial1_spks[5, :]  # 40 time bins
print(f"Spike train of Neuron 6 in Trial 1: {neuron_6_spike_train}")

In [ ]:
# Extract the spike count at time bin 3 (index 2 in Python)
neuron_6_bin_3 = trial1_spks[5, 2]
print(f"Neuron 6 spike count at Time Bin 3: {neuron_6_bin_3}")  
# Expected output: 1 if there was a spike, 0 otherwise

In [ ]:
# Extract the brain area of neuron 6 (index 5 in Python)
neuron_6_brain_area = session1.rx2("brain_area")[5]
print(f"Neuron 6 belongs to brain area: {neuron_6_brain_area}")  
# Expected output: "ACA"

In [ ]:
def get_trial_data(session, trial_id):
    """
    Summarizes neural activity for a given trial:
      - Total spike counts per brain area
      - Number of neurons per brain area
      - Average spike rate per neuron in each brain area
      - Trial metadata (contrast_left, contrast_right, feedback_type)
    """
    spikes = np.array(session.rx2("spks")[trial_id])  # Shape: (n_neurons, n_time_bins)
    neuron_spike = np.sum(spikes, axis=1)  # Sum of spikes over 40 time bins
    brain_area = np.array(session.rx2("brain_area"))  # Brain area for each neuron
    
    df = pd.DataFrame({
        "neuron_spike": neuron_spike,
        "brain_area": brain_area
    })
    
    # Group by brain area and compute summary statistics
    trial_df = (df.groupby("brain_area")
                .agg(region_sum_spike=("neuron_spike", "sum"),
                     region_count=("neuron_spike", "count"),
                     region_mean_spike=("neuron_spike", "mean"))
                .reset_index())

    # Add trial metadata
    trial_df["trial_id"] = trial_id + 1  # 1-based indexing
    trial_df["contrast_left"] = session.rx2("contrast_left")[trial_id]
    trial_df["contrast_right"] = session.rx2("contrast_right")[trial_id]
    trial_df["feedback_type"] = session.rx2("feedback_type")[trial_id]
    
    return trial_df

# Example: Get data for trial 17 in session 1
trial_summary = get_trial_data(session1_data, trial_id=17)
print(trial_summary)

In [ ]:
def get_trial_bin_average(session):
    """
    Computes the average spike rate per time bin for each trial.
    Each row represents a trial, and each column represents a time bin.
    """
    spks = session.rx2('spks')
    trial_avgs = []
    n_trials = len(spks)
    n_bins = np.array(spks[0]).shape[1]  # Assuming all trials have the same number of bins

    for trial_id in range(n_trials):
        trial_matrix = np.array(spks[trial_id])  # (n_neurons, n_bins)
        bin_avg = np.mean(trial_matrix, axis=0)  # Average across neurons
        trial_avgs.append(bin_avg)

    bin_columns = [f"bin{i+1}" for i in range(n_bins)]
    df = pd.DataFrame(trial_avgs, columns=bin_columns)

    # Add trial metadata
    df["trial_id"] = np.arange(1, n_trials + 1)
    df["contrast_left"] = np.array(session.rx2("contrast_left"))
    df["contrast_right"] = np.array(session.rx2("contrast_right"))
    df["feedback_type"] = np.array(session.rx2("feedback_type"))
    return df

# Example: Get trial bin averages for session 1
trial_bin_avg_df = get_trial_bin_average(session1_data)
print(trial_bin_avg_df.head())

In [ ]:
len(session.rx2('spks'))

In [ ]:
def get_session_summary(all_sessions):
    """
    Computes summary statistics for each session:
    - Number of neurons
    - Number of unique brain areas
    - Average spike rate
    - Success rate
    """
    session_summaries = []

    for session_id, session in enumerate(all_sessions, start=1):
        spks = session.rx2("spks")
        brain_areas = np.array(session.rx2("brain_area"))
        feedback = np.array(session.rx2("feedback_type"))
        n_neurons = len(brain_areas)
        n_brain_areas = len(np.unique(brain_areas))

        # Compute mean session spike rate
        mean_spike_rate = np.mean([np.sum(np.array(trial)) / np.array(trial).size for trial in spks])

        # Compute success rate
        success_rate = np.mean(feedback == 1)

        session_summaries.append({
            "session_id": session_id,
            "n_neurons": n_neurons,
            "n_brain_areas": n_brain_areas,
            "mean_spike_rate": mean_spike_rate,
            "success_rate": success_rate
        })

    return pd.DataFrame(session_summaries)

# Compute session summaries
session_summary_df = get_session_summary(all_sessions)
print(session_summary_df)

In [ ]:
import seaborn as sns

plt.figure(figsize=(10, 5))
sns.barplot(x="session_id", y="n_neurons", data=session_summary_df, palette="coolwarm")
plt.xlabel("Session ID")
plt.ylabel("Number of Neurons")
plt.title("Number of Neurons in Each Session")
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x="session_id", y="n_brain_areas", data=session_summary_df, palette="viridis")
plt.xlabel("Session ID")
plt.ylabel("Number of Brain Areas")
plt.title("Number of Brain Areas in Each Session")
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x="session_id", y="mean_spike_rate", data=session_summary_df, palette="magma")
plt.xlabel("Session ID")
plt.ylabel("Average Spike Rate")
plt.title("Average Spike Rate per Session")
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(x="session_id", y="success_rate", data=session_summary_df, palette="Blues")
plt.xlabel("Session ID")
plt.ylabel("Success Rate")
plt.title("Success Rate per Session")
plt.show()

In [ ]:
def get_contrast_summary(all_sessions):
    """
    Computes contrast differences and success rates for all trials across sessions.
    """
    contrast_data = []

    for session_id, session in enumerate(all_sessions, start=1):
        contrast_left = np.array(session.rx2("contrast_left"))
        contrast_right = np.array(session.rx2("contrast_right"))
        feedback = np.array(session.rx2("feedback_type"))  # 1 for success, -1 for failure

        contrast_diff = np.abs(contrast_left - contrast_right)

        for i in range(len(contrast_diff)):
            contrast_data.append({
                "session_id": session_id,
                "contrast_diff": contrast_diff[i],
                "success": 1 if feedback[i] == 1 else 0  # Convert feedback to binary
            })

    return pd.DataFrame(contrast_data)

# Compute contrast summary
contrast_df = get_contrast_summary(all_sessions)

# Print summary
print(contrast_df.head())

In [ ]:
print("Number of NaN values:", contrast_df["contrast_diff"].isna().sum())
print("Total entries:", len(contrast_df))

In [ ]:
print("Data type:", contrast_df["contrast_diff"].dtype)
# Convert to float if needed
contrast_df["contrast_diff"] = pd.to_numeric(contrast_df["contrast_diff"], errors="coerce")

In [ ]:
plt.figure(figsize=(8, 5))
sns.histplot(contrast_df["contrast_diff"], bins=20, kde=True, color="purple")
plt.xlabel("Contrast Difference")
plt.ylabel("Count")
plt.title("Distribution of Contrast Differences Across Trials")
plt.show()

In [ ]:
# Compute success rate for each contrast difference level
contrast_success_rate = (contrast_df
                         .groupby("contrast_diff")["success"]
                         .mean()
                         .reset_index()
                         .sort_values("contrast_diff"))

# Plot success rate vs contrast difference
plt.figure(figsize=(8, 5))
sns.lineplot(x="contrast_diff", y="success", data=contrast_success_rate, marker="o", color="blue")
plt.xlabel("Contrast Difference")
plt.ylabel("Success Rate")
plt.title("Success Rate vs Contrast Difference")
plt.ylim(0, 1)  # Ensure y-axis is between 0 and 1
plt.grid(True)
plt.show()

In [ ]:
# Merge contrast_df with session metadata
mouse_data = []
for session_id, session in enumerate(all_sessions, start=1):
    mouse_name = session.rx2("mouse_name")[0]
    contrast_values = contrast_df.loc[contrast_df["session_id"] == session_id, "contrast_diff"].values
    for contrast in contrast_values:
        mouse_data.append({
            "session_id": session_id,
            "mouse_name": mouse_name,
            "contrast_diff": contrast
        })

mouse_contrast_df = pd.DataFrame(mouse_data)

# Plot contrast distribution per mouse
plt.figure(figsize=(10, 6))
sns.boxplot(x="mouse_name", y="contrast_diff", data=mouse_contrast_df, palette="Set2")
plt.xlabel("Mouse Name")
plt.ylabel("Contrast Difference")
plt.title("Distribution of Contrast Differences by Mouse")
plt.grid(True)
plt.show()

In [ ]:
print(mouse_contrast_df["mouse_name"].value_counts())

In [ ]:
print(mouse_contrast_df.isna().sum())
print(mouse_contrast_df["contrast_diff"].describe())

In [ ]:
print(contrast_df.isna().sum())
print(contrast_df[contrast_df["contrast_diff"].isna()].head())

In [ ]:
# Check original contrast_left and contrast_right for NaN cases
nan_contrast_rows = contrast_df[contrast_df["contrast_diff"].isna()]
session_ids_with_nan = nan_contrast_rows["session_id"].unique()
print("Sessions with NaN contrast_diff:", session_ids_with_nan)

# Now check their original contrast values
for session_id in session_ids_with_nan:
    session_data = all_sessions[session_id - 1]  # Adjust for 0-based index
    contrast_left = np.array(session_data.rx2("contrast_left"))
    contrast_right = np.array(session_data.rx2("contrast_right"))

    print(f"Session {session_id}:")
    print("Left contrast NaN count:", np.isnan(contrast_left).sum())
    print("Right contrast NaN count:", np.isnan(contrast_right).sum())
    print()

In [ ]:
# Load all sessions (assumes 18 sessions named session1.rds to session18.rds)
all_sessions = []
for i in range(1, 19):
    file_name = f'session{i}.rds'
    file_path = os.path.join(data_path, file_name)
    session_data = read_rds(file_path)
    all_sessions.append(session_data)

# Extract session-level metadata and summary stats
session_summaries = []
for sess in all_sessions:
    mouse_name = sess.rx2('mouse_name')[0]
    date_exp = sess.rx2('date_exp')[0]
    spks = sess.rx2('spks')  # list of spike matrices (one per trial)
    first_trial_spks = np.array(spks[0])
    n_neurons = first_trial_spks.shape[0]         # number of neurons
    n_trials = len(spks)                           # number of trials
    feedback = np.array(sess.rx2('feedback_type'))
    success_rate = np.mean(feedback == 1)          # success rate (1=success, -1=failure)
    brain_area = np.array(sess.rx2('brain_area'))
    n_brain_area = len(np.unique(brain_area))
    
    session_summaries.append({
        "mouse_name": mouse_name,
        "date_exp": date_exp,
        "n_neurons": n_neurons,
        "n_trials": n_trials,
        "n_brain_area": n_brain_area,
        "success_rate": success_rate
    })

session_summary_df = pd.DataFrame(session_summaries)
print(session_summary_df)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Bar plot: Session success rate by date and mouse
plt.figure(figsize=(10, 6))
sns.barplot(x="date_exp", y="success_rate", hue="mouse_name", data=session_summary_df)
plt.title("Session Success Rate by Mouse and Date")
plt.xlabel("Experiment Date")
plt.ylabel("Success Rate")
plt.xticks(rotation=45)
plt.legend(title="Mouse Name")
plt.tight_layout()
plt.show()

# Bar plot: Number of neurons per session by date and mouse
plt.figure(figsize=(10, 6))
sns.barplot(x="date_exp", y="n_neurons", hue="mouse_name", data=session_summary_df)
plt.title("Number of Neurons per Session")
plt.xlabel("Experiment Date")
plt.ylabel("Number of Neurons")
plt.xticks(rotation=45)
plt.legend(title="Mouse Name")
plt.tight_layout()
plt.show()

In [ ]:
# Select one session (e.g., the first session)
session = all_sessions[0]

# Extract spike data (list of matrices, one per trial) and convert to 3D NumPy array:
spks_list = session.rx2('spks')
spks_array = np.array([np.array(trial) for trial in spks_list])
print(f"Spike Data Shape: {spks_array.shape}")  # Expected: (n_trials, n_neurons, n_time_bins)

# Preview the spike matrix for the first trial
print("First trial spike matrix:")
print(spks_array[0])

# Compute percentage of zeros (indicating inactive bins)
zero_percentage = np.mean(spks_array[0] == 0) * 100
print(f"Percentage of zeroes in first trial: {zero_percentage:.2f}%")

In [ ]:
# Plot a heatmap of the first trial's spike matrix
plt.figure(figsize=(10, 8))
ax = sns.heatmap(spks_array[0], cmap="viridis", cbar=True)
ax.figure.colorbar(ax.collections[0], label="Spike Count")
plt.xlabel("Time Bins")
plt.ylabel("Neuron Index")
plt.title("Heatmap of Spike Counts (First Trial)")
plt.show()

In [ ]:
def get_trial_data(session, trial_id):
    """
    For a given session and trial (0-indexed), compute for each brain area:
      - region_sum_spike: total spikes (sum over 40 time bins) for neurons in that area
      - region_count: number of neurons in that area
      - region_mean_spike: average spike count per neuron in that area
    Also attaches trial-level metadata (trial_id, contrast levels, feedback).
    """
    spikes = session.rx2('spks')[trial_id]
    neuron_spike = np.sum(np.array(spikes), axis=1)  # sum over time bins per neuron
    brain_area = np.array(session.rx2('brain_area'))
    
    df = pd.DataFrame({
        "neuron_spike": neuron_spike,
        "brain_area": brain_area
    })
    
    trial_df = (df.groupby("brain_area")
                .agg(region_sum_spike=("neuron_spike", "sum"),
                     region_count=("neuron_spike", "count"),
                     region_mean_spike=("neuron_spike", "mean"))
                .reset_index())
    
    # Append trial metadata
    trial_df["trial_id"] = trial_id + 1  # 1-indexed
    trial_df["contrast_left"] = session.rx2("contrast_left")[trial_id]
    trial_df["contrast_right"] = session.rx2("contrast_right")[trial_id]
    trial_df["feedback_type"] = session.rx2("feedback_type")[trial_id]
    
    return trial_df

# Example: Get region summary for trial 2 of session 1
trial_summary = get_trial_data(session, trial_id=1)
print(trial_summary)

In [ ]:
def get_trial_bin_average(session):
    """
    For each trial in a session, compute the average spike count per time bin 
    (averaging over all neurons). Returns a DataFrame with columns bin1...bin40,
    plus trial-level metadata.
    """
    spks = session.rx2('spks')
    trial_avgs = []
    n_trials = len(spks)
    # Assuming all trials have the same number of time bins:
    n_bins = np.array(spks[0]).shape[1]
    
    for trial_id in range(n_trials):
        trial_matrix = np.array(spks[trial_id])  # shape: (n_neurons, n_bins)
        bin_avg = np.mean(trial_matrix, axis=0)    # average per time bin
        trial_avgs.append(bin_avg)
    
    bin_columns = [f"bin{i+1}" for i in range(n_bins)]
    df = pd.DataFrame(trial_avgs, columns=bin_columns)
    
    # Add trial metadata
    df["trial_id"] = np.arange(1, n_trials + 1)
    df["contrast_left"] = np.array(session.rx2("contrast_left"))
    df["contrast_right"] = np.array(session.rx2("contrast_right"))
    df["feedback_type"] = np.array(session.rx2("feedback_type"))
    return df

# Example: Get trial bin averages for session 1
trial_bin_avg_df = get_trial_bin_average(session)
print(trial_bin_avg_df.head())

In [ ]:
# For a chosen session (e.g., session 1), compute contrast difference
session1 = all_sessions[0]
contrast_left = np.array(session1.rx2("contrast_left"))
contrast_right = np.array(session1.rx2("contrast_right"))
contrast_diff = np.abs(contrast_left - contrast_right)

df_contrast = pd.DataFrame({
    "trial_id": np.arange(1, len(contrast_diff)+1),
    "contrast_diff": contrast_diff,
    "feedback_type": np.array(session1.rx2("feedback_type"))
})

# Plot distribution of contrast differences
plt.figure(figsize=(6, 4))
sns.histplot(data=df_contrast, x="contrast_diff", bins=np.arange(0, 1.25, 0.25), kde=False, color="skyblue", edgecolor="black")
plt.xlabel("Contrast Difference")
plt.ylabel("Number of Trials")
plt.title("Contrast Difference Distribution (Session 1)")
plt.show()

# Compute success rate (feedback==1) by contrast difference
df_contrast["success"] = (df_contrast["feedback_type"] == 1).astype(int)
success_rate_by_contrast = df_contrast.groupby("contrast_diff")["success"].mean().reset_index()

plt.figure(figsize=(6, 4))
sns.lineplot(data=success_rate_by_contrast, x="contrast_diff", y="success", marker="o")
plt.xlabel("Contrast Difference")
plt.ylabel("Success Rate")
plt.title("Success Rate by Contrast Difference (Session 1)")
plt.ylim(0, 1)
plt.show()

In [ ]:
def success_rate_over_time(session, bin_size=25):
    """
    Bins trials into groups (e.g., 25 trials per bin) and computes the
    success rate (proportion of feedback==1) per bin.
    """
    feedback = np.array(session.rx2("feedback_type"))
    df = pd.DataFrame({
        "trial_id": np.arange(1, len(feedback)+1),
        "feedback": feedback
    })
    df["bin"] = ((df["trial_id"] - 1) // bin_size) + 1
    summary = df.groupby("bin")["feedback"].apply(lambda x: np.mean(x == 1)).reset_index()
    summary.rename(columns={"feedback": "success_rate"}, inplace=True)
    return summary

sr_session1 = success_rate_over_time(session, bin_size=25)
plt.figure(figsize=(8, 4))
sns.lineplot(data=sr_session1, x="bin", y="success_rate", marker="o")
plt.xlabel("Bin (25 Trials per Bin)")
plt.ylabel("Success Rate")
plt.title("Success Rate Change Over Time (Session 1)")
plt.ylim(0, 1)
plt.show()

In [ ]:
def overall_spike_rate(session):
    """
    For each trial in a session, compute the average spike rate per neuron.
    (i.e. total spikes in the trial divided by the number of neurons)
    """
    spks = session.rx2("spks")
    rates = [np.mean(np.sum(np.array(trial), axis=1)) for trial in spks]
    df = pd.DataFrame({
        "trial_id": np.arange(1, len(rates) + 1),
        "avg_spike_rate": rates
    })
    return df

spike_rate_df = overall_spike_rate(session)
plt.figure(figsize=(8, 4))
sns.lineplot(data=spike_rate_df, x="trial_id", y="avg_spike_rate", marker="o")
plt.xlabel("Trial ID")
plt.ylabel("Average Spike Rate (per Neuron)")
plt.title("Overall Neuron Spike Rate Over Trials (Session 1)")
plt.show()

In [ ]:
from sklearn.decomposition import PCA

# Use the trial bin averages from session 1
bin_features = trial_bin_avg_df.filter(regex="^bin").values

# Perform PCA (reduce to 2 components)
pca = PCA(n_components=2)
pc = pca.fit_transform(bin_features)

pca_df = pd.DataFrame(pc, columns=["PC1", "PC2"])
pca_df["trial_id"] = trial_bin_avg_df["trial_id"]

plt.figure(figsize=(8, 6))
sns.scatterplot(data=pca_df, x="PC1", y="PC2", hue="trial_id", palette="viridis", legend=False)
plt.title("PCA of Trial Bin Averages (Session 1)")
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% variance)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% variance)")
plt.show()

In [ ]:
def get_trial_bin_average(session):
    spks = session.rx2("spks")
    trial_avgs = []
    n_trials = len(spks)
    n_bins = np.array(spks[0]).shape[1]
    
    for trial_id in range(n_trials):
        trial_matrix = np.array(spks[trial_id])
        bin_avg = np.mean(trial_matrix, axis=0)
        trial_avgs.append(bin_avg)
        
    bin_columns = [f"bin{i+1}" for i in range(n_bins)]
    df = pd.DataFrame(trial_avgs, columns=bin_columns)
    df["trial_id"] = np.arange(1, n_trials+1)
    df["contrast_left"] = np.array(session.rx2("contrast_left"))
    df["contrast_right"] = np.array(session.rx2("contrast_right"))
    df["feedback_type"] = np.array(session.rx2("feedback_type"))
    # Compute contrast difference
    df["contrast_diff"] = np.abs(df["contrast_left"] - df["contrast_right"])
    return df

trial_dfs = []
for idx, sess in enumerate(all_sessions):
    df_trial = get_trial_bin_average(sess)
    df_trial["session_id"] = idx + 1
    df_trial["mouse_name"] = sess.rx2("mouse_name")[0]
    trial_dfs.append(df_trial)

# Combine trials from all sessions into one DataFrame
full_functional_df = pd.concat(trial_dfs, ignore_index=True)
print(full_functional_df.head())

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

# Define the predictive features.
# Here we use session_id, trial_id, contrast_left, contrast_right, contrast_diff, and all bin features.
bin_cols = [col for col in full_functional_df.columns if col.startswith("bin")]
predictive_features = ["session_id", "trial_id", "contrast_left", "contrast_right", "contrast_diff"] + bin_cols

# Prepare feature matrix X and target y
bin_cols = [col for col in full_functional_df.columns if col.startswith("bin")]
predictive_features = ["session_id", "trial_id", "contrast_left", "contrast_right", "contrast_diff"] + bin_cols

X = full_functional_df[predictive_features].copy()
X["session_id"] = pd.to_numeric(X["session_id"])
X["trial_id"] = pd.to_numeric(X["trial_id"])
y = full_functional_df["feedback_type"]

# Convert y: map -1 -> 0 and keep 1 as 1
y = np.where(y == -1, 0, 1)

# Split into training and unused parts
from sklearn.model_selection import train_test_split
X_train, X_unused, y_train, y_unused = train_test_split(X, y, test_size=0.2, random_state=123)

# Train an XGBoost classifier.
from xgboost import XGBClassifier
model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
model.fit(X_train, y_train)

print("Model training complete.")